In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Exploratory & Statistical Analysis


### Task Details
You're a marketing analyst and you've been told by the Chief Marketing Officer that recent marketing campaigns have not been as effective as they were expected to be. You need to analyze the data set to understand this problem and propose data-driven solutions.

### Expected Submission
Submit a well documented notebook with these three sections:

### Section 01: Exploratory Data Analysis
* Are there any null values or outliers? How will you wrangle/handle them?
* Are there any variables that warrant transformations?
* Are there any useful variables that you can engineer with the given data?
* Do you notice any patterns or anomalies in the data? Can you plot them?
### Section 02: Statistical Analysis

Please run statistical tests in the form of regressions to answer these questions & propose data-driven action recommendations to your CMO. Make sure to interpret your results * with non-statistical jargon so your CMO can understand your findings.

* What factors are significantly related to the number of store purchases?
* Does US fare significantly better than the Rest of the World in terms of total purchases?
* Your supervisor insists that people who buy gold are more conservative. Therefore, people who spent an above average amount on gold in the last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test
* Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish? What other factors are * significantly related to amount spent on fish? (Hint: use your knowledge of interaction variables/effects)
* Is there a significant relationship between geographical regional and success of a campaign?
### Section 03: Data Visualization
Please plot and visualize the answers to the below questions.

* Which marketing campaign is most successful?
* What does the average customer look like for this company?
* Which products are performing best?
* Which channels are underperforming?
Evaluation
* This is not a formal competition, so results won't be measured using a strict metric. Rather, what one would like to see is a well-defined process of exploratory and statistical analysis with insightful conclusions.


## Preparing the data

In [ ]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import matplotlib as mpl

colors = ['#05445E','#189AB4','#75E6DA','#9DAFB0', '#4D707E', '#56B66B']

mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right']= False
mpl.rcParams['axes.spines.left']= False
mpl.rcParams['axes.spines.bottom'] = True

In [ ]:
data = pd.read_csv("/kaggle/input/marketing-data/marketing_data.csv")
data.sample(5)

In [ ]:
data.columns

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data = data.rename(columns={' Income ' : "Income"})

# Section 01

## Feature Engineering

Looking at the data there are several column that can we process to create new column

* Age : can be derived fron Year_Birth
* Age Group : can be derived from Age
* Totalchild : sum of kidhome and Teenhome

and there are also column that need to be pre-process that is 
* Income : change into float
* Dt_Customer : change into pandas Datetime

In [ ]:
# clean income 
def clean_income(row) : 
    income = re.findall('\d+,\d+', row)
    income = re.sub('\W', '', income)
    print(income)

    return income 

def get_age_group(age) : 
    if 18 <= age <= 35 : 
        return 'Millenial' 
    elif 35 < age <= 50 : 
        return 'Gen X'
    elif 50 < age <= 70 : 
        return 'Boomer'
    else : 
        return 'Silent'

data['Age'] = 2020 - data['Year_Birth']
data['Income'] = data['Income'].replace('[\$,]', '', regex=True).astype(float)
data['Age Group'] = data['Age'].apply(get_age_group)
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])
data['Totalchild'] = data['Kidhome'] + data['Teenhome']

To make my exploration easier i divided our data into multiple categories that is 

* behavioral 
* demographics 
* psychographics 
* geographics 
* products

In [ ]:
col_behavioral = [
    'MntWines', 
    'MntFruits', 
    'MntMeatProducts', 
    'MntFishProducts', 
    'MntSweetProducts', 
    'MntGoldProds', 
    'NumDealsPurchases', 
    'NumWebPurchases', 
    'NumCatalogPurchases', 
    'NumStorePurchases', 
    'NumWebVisitsMonth', 
    'Recency'
]

col_demographics = [
    'Year_Birth', 
    'Education', 
    'Marital_Status', 
    'Income', 
    'Kidhome', 
    'Teenhome', 
    'Dt_Customer'
] 

col_psychographics = [
'AcceptedCmp1', 
'AcceptedCmp2', 
'AcceptedCmp3', 
'AcceptedCmp4', 
'AcceptedCmp5', 
'Response', 
'Complain'

]

col_geographics = ['Country']

col_products = [
'MntWines', 
'MntFruits', 
'MntMeatProducts', 
'MntFishProducts', 
'MntSweetProducts', 
'MntGoldProds'
]

## Are there any null values or outliers? How will you wrangle/handle them?

looking at this data, i found that there are several null values in feature "Income"

since we only have 24 rows with missing value, i will just remove those rows

In [ ]:
data.isnull().sum()


In [ ]:
# drop missing value 
data = data.dropna()

In [ ]:
# columns to be investigated
col_demographics

In [ ]:
data.groupby('Marital_Status').count()['ID']

On Marital Status Column, we can see that we have an anomalies. "Absurd", "Alone" and "YOLO" is not a marital status, at least by my understanding. Since the anomalies is responsible for only 7 rows, i decided to drop those rows

In [ ]:
data = data[~data['Marital_Status'].isin(['Absurd', 'Alone', 'YOLO'])]

In [ ]:
df = data['ID'].groupby(data['Education']).count()

# Pie chart
#colors
explode = (0.05,0.05,0.05,0.05,0.05)
 
fig1, ax1 = plt.subplots(figsize=(8, 5))
patches, texts, autotexts = ax1.pie(df.values, colors = colors, labels=df.index, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

for text in texts:
    text.set_color('#131313')
for autotext in autotexts:
    autotext.set_color('#131313')

# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

In [ ]:
df = data['ID'].groupby(data['Marital_Status']).count()
df = df.sort_values()[-5:]

# Pie chart
#colors
explode = (0.05,0.05,0.05,0.05,0.05)
 
fig1, ax1 = plt.subplots(figsize=(8, 5))
patches, texts, autotexts = ax1.pie(df.values, colors = colors, labels=df.index, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

for text in texts:
    text.set_color('#131313')
for autotext in autotexts:
    autotext.set_color('#131313')

# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

In [ ]:
fig1, ax = plt.subplots(1, 2, figsize=(16, 5))

sns.kdeplot(data['Age'], ax=ax[0], shade=True, color=colors[0])

ax[0].set_title("Age Density", fontsize=14, fontweight='bold')
ax[0].set_ylabel('Density', fontsize=13)
ax[0].set_xlabel('Age', fontsize=13)
ax[0].yaxis.grid(alpha=0.33)
ax[0].set_axisbelow(True)

sns.kdeplot(data['Income'], ax=ax[1], shade=True, color=colors[0])

ax[1].set_title("Income Density", fontsize=14, fontweight='bold')
ax[1].set_ylabel('Density', fontsize=13)
ax[1].set_xlabel('Income', fontsize=13)
ax[1].yaxis.grid(alpha=0.33)
ax[1].set_axisbelow(True)

we can see that there is an anomaly in Age and an outlier in Income, first we are going to investigate further Age column

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

sns.boxplot(data['Age'], ax=ax, palette=colors)

ax.xaxis.grid(alpha=0.33)
ax.set_axisbelow(True)

ax.set_title("Age Boxplot", fontsize=14, fontweight='bold')
ax.set_xlabel('Age', fontsize=13)

Here we detect that there are 3 people with Age anomalies, those people is over than 120 years old! which is highly unlikely. Since it is only 3 people we will solve this problem by drop those 3 people on our data

In [ ]:
# remove Age outlier
data = data[data['Age'] < 120]

now we are going to investigate further our income columns

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))

sns.boxplot(data['Income'], ax=ax, palette=colors)

ax.xaxis.grid(alpha=0.33)
ax.set_axisbelow(True)

ax.set_title("Income Boxplot", fontsize=14, fontweight='bold')
ax.set_xlabel('Income', fontsize=13)

Here we see two anomalies, person with income on range 100.000 to 200.000 and person with income of  600.000 . 

While the first one is still possible, the second one however it is highly unlikely a person having an income over 600.000$

to solve this problem i decided to drop the people with income over 600.000$

In [ ]:
# remove Income outlier
data = data[data['Income'] < 200000]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14,6))

sns.boxplot(data['Age'], ax=ax[0], palette=colors)

ax[0].xaxis.grid(alpha=0.33)
ax[0].set_axisbelow(True)

ax[0].set_title("Age Boxplot", fontsize=14, fontweight='bold')
ax[0].set_xlabel('Age', fontsize=13)

sns.boxplot(data['Income'], ax=ax[1], palette=colors)

ax[1].xaxis.grid(alpha=0.33)
ax[1].set_axisbelow(True)

ax[1].set_title("Income Boxplot", fontsize=14, fontweight='bold')
ax[1].set_xlabel('Income', fontsize=13)

### Products

now we are going to investigate whether there are outlier/anomalies on Products columns

In [ ]:
df = data[col_products]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.boxplot(data=df, palette=colors)

ax.set_yscale('log')
ax.set_title("Products Boxplot", fontsize=14, fontweight='bold')
ax.set_xlabel('Products Bought', fontsize=13)
ax.set_ylabel('Ammount of Products Purchased (log scaled)', fontsize=13)
ax.yaxis.grid(alpha=0.33)
ax.set_axisbelow(True)

Although if we look at the chart we can see an outlier for every Products, but in my opinion i think it is not a true outlier. We cannot judge all customers to buy an amount of goods that can be said to be "normal", for example: a customer may be a merchant who needs a lot of goods

So in this case i'll leave it as it is

In [ ]:
df = data[col_products].sum()

# Pie chart
#colors
explode = (0.05,0.05,0.05,0.05,0.05,0.05)
 
fig1, ax1 = plt.subplots(figsize=(8, 5))
patches, texts, autotexts = ax1.pie(df.values, colors = colors, labels=df.index, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

for text in texts:
    text.set_color('#131313')
for autotext in autotexts:
    autotext.set_color('#131313')

# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

## Do you notice any patterns or anomalies in the data? Can you plot them?

### Customer Generation pattern on the market

In [ ]:
df = data['ID'].groupby(data['Age Group']).count()
# Pie chart
#colors
explode = (0.05,0.05,0.05,0.05)
 
fig1, ax1 = plt.subplots(figsize=(8, 5))
patches, texts, autotexts = ax1.pie(df.values, colors = colors, labels=df.index, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
#draw circle
centre_circle = plt.Circle((0,0),0.50,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

for text in texts:
    text.set_color('#131313')
for autotext in autotexts:
    autotext.set_color('#131313')

# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
plt.show()

Market is more dominated with person on Boomer Generation (43.2%) and Generation X (42.1%)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

df = data[col_products+['Age']]

bins = [18, 35, 50, 70, 150]

labels = ['Millenial', 'Gen X', 'Boomer', 'Silent']
df['Age_Group'] = pd.cut(df.Age, bins=bins, labels=labels)
df = df.groupby('Age_Group').sum().reset_index()
df.plot(x='Age_Group', y=col_products, kind='bar', ax=ax, color=colors, width=0.8, edgecolor='#131313')

ax.set_title("Customer Products Behaviour Per Age Group (Total)", fontsize=14, fontweight='bold')
ax.set_ylabel('Total Products Purchased', fontsize=13)
ax.set_xlabel('Age Group', fontsize=13)
ax.yaxis.grid(alpha=0.33)
ax.set_axisbelow(True)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

df = data[col_products+['Age']]

bins = [18, 35, 50, 70, 150]

labels = ['Millenial', 'Gen X', 'Boomer', 'Silent']
df['Age_Group'] = pd.cut(df.Age, bins=bins, labels=labels)
df = df.groupby('Age_Group').mean().reset_index()
df.plot(x='Age_Group', y=col_products, kind='bar', ax=ax, color=colors, width=0.8, edgecolor='#131313')

ax.set_title("Customer Products Behaviour Per Age Group (Average)", fontsize=14, fontweight='bold')
ax.set_ylabel('Average Products Purchased', fontsize=13)
ax.set_xlabel('Age Group', fontsize=13)
ax.yaxis.grid(alpha=0.33)
ax.set_axisbelow(True)

### Customer Income Pattern on Market

In [ ]:
fig = plt.figure(figsize=(14, 18))

gs = fig.add_gridspec(3, 2) 

df_columns = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df = data
ax = [None for i in range(len(df_columns))]
colors = ['#05445E','#189AB4','#75E6DA','#9DAFB0', '#4D707E']

ax[0] = fig.add_subplot(gs[0, 0]) 
sns.scatterplot(data=df, x='Income', y='MntWines', hue='Education', ax=ax[0], palette=colors)

ax[1] = fig.add_subplot(gs[0, 1]) 
sns.scatterplot(data=df, x='Income', y='MntFruits', hue='Education', ax=ax[1], palette=colors)

ax[2] = fig.add_subplot(gs[1, 0]) 
sns.scatterplot(data=df, x='Income', y='MntMeatProducts', hue='Education', ax=ax[2], palette=colors)

ax[3] = fig.add_subplot(gs[1, 1]) 
sns.scatterplot(data=df, x='Income', y='MntFishProducts', hue='Education', ax=ax[3], palette=colors)

ax[4] = fig.add_subplot(gs[2, 0]) 
sns.scatterplot(data=df, x='Income', y='MntSweetProducts', hue='Education', ax=ax[4], palette=colors)

ax[5] = fig.add_subplot(gs[2, 1]) 
sns.scatterplot(data=df, x='Income', y='MntGoldProds', hue='Education', ax=ax[5], palette=colors)


for ix in range(len(df_columns)) : 
    ax[ix].set_title(f"Customer Purchase Behaviour based on income", fontsize=13, fontweight='bold')
    ax[ix].set_ylabel(f'{df_columns[ix]}')
    ax[ix].yaxis.grid(alpha=0.33)
    ax[ix].xaxis.grid(alpha=0.33)
    ax[ix].set_xlabel('Income (Log Scaled)')
    ax[ix].set_xscale('log')
    ax[ix].set_axisbelow(True)


From this graph we can see that the more Income a person has, the more products that they will buy, the number of products tends to increase exponentially

### Offline vs Online Purchases

In [ ]:
df_columns = ['NumStorePurchases', 'NumWebPurchases']

fig, ax = plt.subplots(figsize=(8,6))
df = data[df_columns].sum()
sns.barplot(df.index, df.values, palette=colors, ax=ax, edgecolor='#131313')

ax.set_title("Where Customer Purchase Goods", fontsize=14, fontweight='bold')
ax.set_ylabel('Total Products Purchased', fontsize=13)
ax.set_xlabel('Place of Purchases', fontsize=13)
ax.yaxis.grid(alpha=0.7)
ax.set_axisbelow(True)

Most of Customer buy the product offline rather than online

In [ ]:
df_columns = ['NumStorePurchases', 'NumWebPurchases']

df = data[df_columns].groupby(data['Age Group']).mean()

fig, ax = plt.subplots(figsize=(12,6))

df.T.plot(kind='bar', color=colors, width=0.8, edgecolor='#131313', ax=ax)

ax.set_title("Where Customer Purchase Goods per Age Group", fontsize=14, fontweight='bold')
ax.set_ylabel('Average Products Purchased', fontsize=13)
ax.set_xlabel('Place of Purchases', fontsize=13)
ax.yaxis.grid(alpha=0.7)
ax.set_axisbelow(True)

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))


To be continued
